In [7]:
import numpy as np
import pandas as pd
from datetime import datetime
import talib
from numba import njit

import vectorbt as vbt
from vectorbt.utils.colors import adjust_opacity
from vectorbt.utils.enum_ import map_enum_fields
from vectorbt.base.reshape_fns import broadcast, flex_select_auto_nb, to_2d_array
from vectorbt.portfolio.enums import SizeType, Direction, NoOrder, OrderStatus, OrderSide
from vectorbt.portfolio import nb

In [8]:
# Fetch price history
symbols = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'BNB-USD', 'BCH-USD', 'LTC-USD']
start = '2020-01-01 UTC'  # crypto is UTC
end = '2020-09-01 UTC'
# OHLCV by column
ohlcv = vbt.YFData.download(symbols, start=start, end=end).concat()


這邊已經拿完資料了
data內的資料: {'BTC-USD':                                    Open          High           Low  \
Date                                                                  
2020-01-01 00:00:00+00:00   7194.892090   7254.330566   7174.944336   
2020-01-02 00:00:00+00:00   7202.551270   7212.155273   6935.270020   
2020-01-03 00:00:00+00:00   6984.428711   7413.715332   6914.996094   
2020-01-04 00:00:00+00:00   7345.375488   7427.385742   7309.514160   
2020-01-05 00:00:00+00:00   7410.451660   7544.497070   7400.535645   
...                                 ...           ...           ...   
2020-08-27 00:00:00+00:00  11485.608398  11570.786133  11185.941406   
2020-08-28 00:00:00+00:00  11325.295898  11545.615234  11316.422852   
2020-08-29 00:00:00+00:00  11541.054688  11585.640625  11466.292969   
2020-08-30 00:00:00+00:00  11508.713867  11715.264648  11492.381836   
2020-08-31 00:00:00+00:00  11713.306641  11768.876953  11598.318359   

                                  Close     

/home/abcd/桌面/mydir/backtesting/lib/python3.8/site-packages/vectorbt/data/base.py:695: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_data[c].loc[:, s] = col_data
/home/abcd/桌面/mydir/backtesting/lib/python3.8/site-packages/vectorbt/data/base.py:695: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_data[c].loc[:, s] = col_data
/home/abcd/桌面/mydir/backtesting/lib/python3.8/site-packages/vectorbt/data/base.py:695: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new arra

In [9]:
print(type(ohlcv))
ohlcv['Close']

<class 'dict'>


symbol,BTC-USD,ETH-USD,XRP-USD,BNB-USD,BCH-USD,LTC-USD
Date,,,,,,
2020-01-01 00:00:00+00:00,7200.174316,130.802002,0.192667,13.689083,204.397537,42.017719
2020-01-02 00:00:00+00:00,6985.470215,127.410179,0.188043,13.027011,195.698563,39.823013
2020-01-03 00:00:00+00:00,7344.884277,134.171707,0.193521,13.660452,222.412979,42.415573
2020-01-04 00:00:00+00:00,7410.656738,135.069366,0.194355,13.891512,226.018692,43.326607
2020-01-05 00:00:00+00:00,7411.317383,136.276779,0.195537,14.111019,224.096527,43.553207
...,...,...,...,...,...,...
2020-08-27 00:00:00+00:00,11323.397461,382.632629,0.264511,23.080753,264.584320,56.041016
2020-08-28 00:00:00+00:00,11542.500000,395.874664,0.272046,23.133812,268.914703,57.354404
2020-08-29 00:00:00+00:00,11506.865234,399.921478,0.274497,23.011272,268.954285,57.291733


In [10]:
result = pd.DataFrame.vbt.empty_like(ohlcv['Open'], fill_value=0.)
result

symbol,BTC-USD,ETH-USD,XRP-USD,BNB-USD,BCH-USD,LTC-USD
Date,,,,,,
2020-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-02 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-03 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-04 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-05 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2020-08-27 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-08-28 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-08-29 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
for pattern in talib.get_function_groups()['Pattern Recognition']:
    PRecognizer = vbt.IndicatorFactory.from_talib(pattern)
    pr = PRecognizer.run(ohlcv['Open'], ohlcv['High'], ohlcv['Low'], ohlcv['Close'])
    result = result + pr.integer

In [12]:
result

symbol,BTC-USD,ETH-USD,XRP-USD,BNB-USD,BCH-USD,LTC-USD
Date,,,,,,
2020-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-02 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-03 00:00:00+00:00,100.0,0.0,100.0,0.0,0.0,0.0
2020-01-04 00:00:00+00:00,100.0,0.0,100.0,0.0,0.0,0.0
2020-01-05 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2020-08-27 00:00:00+00:00,0.0,-200.0,0.0,0.0,100.0,0.0
2020-08-28 00:00:00+00:00,400.0,500.0,100.0,300.0,200.0,200.0
2020-08-29 00:00:00+00:00,-400.0,100.0,0.0,-200.0,0.0,100.0


In [13]:
result = result.vbt.fshift(1)
result

symbol,BTC-USD,ETH-USD,XRP-USD,BNB-USD,BCH-USD,LTC-USD
Date,,,,,,
2020-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-03 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-04 00:00:00+00:00,100.0,0.0,100.0,0.0,0.0,0.0
2020-01-05 00:00:00+00:00,100.0,0.0,100.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2020-08-27 00:00:00+00:00,100.0,200.0,200.0,100.0,100.0,100.0
2020-08-28 00:00:00+00:00,0.0,-200.0,0.0,0.0,100.0,0.0
2020-08-29 00:00:00+00:00,400.0,500.0,100.0,300.0,200.0,200.0


In [14]:
size = result / ohlcv['Open']
print(size)
print(size.columns)
size['BTC-USD']

symbol                      BTC-USD   ETH-USD      XRP-USD    BNB-USD  \
Date                                                                    
2020-01-01 00:00:00+00:00       NaN       NaN          NaN        NaN   
2020-01-02 00:00:00+00:00  0.000000  0.000000     0.000000   0.000000   
2020-01-03 00:00:00+00:00  0.000000  0.000000     0.000000   0.000000   
2020-01-04 00:00:00+00:00  0.013614  0.000000   516.739804   0.000000   
2020-01-05 00:00:00+00:00  0.013494  0.000000   514.490612   0.000000   
...                             ...       ...          ...        ...   
2020-08-27 00:00:00+00:00  0.008707  0.517317   721.084531   4.492753   
2020-08-28 00:00:00+00:00  0.000000 -0.522699     0.000000   0.000000   
2020-08-29 00:00:00+00:00  0.034659  1.263623   367.634914  12.967913   
2020-08-30 00:00:00+00:00 -0.034756  0.250240     0.000000  -8.692228   
2020-08-31 00:00:00+00:00  0.034149  0.933469  1413.103041   8.457426   

symbol                      BCH-USD   LTC-USD  
Da

Date
2020-01-01 00:00:00+00:00         NaN
2020-01-02 00:00:00+00:00    0.000000
2020-01-03 00:00:00+00:00    0.000000
2020-01-04 00:00:00+00:00    0.013614
2020-01-05 00:00:00+00:00    0.013494
                               ...   
2020-08-27 00:00:00+00:00    0.008707
2020-08-28 00:00:00+00:00    0.000000
2020-08-29 00:00:00+00:00    0.034659
2020-08-30 00:00:00+00:00   -0.034756
2020-08-31 00:00:00+00:00    0.034149
Freq: D, Name: BTC-USD, Length: 244, dtype: float64

In [15]:
# Simulate portfolio
pf = vbt.Portfolio.from_orders(
    ohlcv['Close'], size, price=ohlcv['Open'],
    init_cash='autoalign', fees=0.001, slippage=0.001)

準備進入計算前的參數
實例已經準備完成


In [16]:
order_df = pf.orders.records_readable
order_df[order_df['Column'] == 'BTC-USD']

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,BTC-USD,2020-01-04 00:00:00+00:00,0.013614,7352.720864,0.1001,Buy
1,1,BTC-USD,2020-01-05 00:00:00+00:00,0.013494,7417.862112,0.1001,Buy
2,2,BTC-USD,2020-01-11 00:00:00+00:00,0.012252,8170.353109,0.1001,Buy
3,3,BTC-USD,2020-01-13 00:00:00+00:00,0.024421,8197.961745,0.2002,Buy
4,4,BTC-USD,2020-01-14 00:00:00+00:00,0.012284,8132.792172,0.0999,Sell
...,...,...,...,...,...,...,...
179,179,BTC-USD,2020-08-26 00:00:00+00:00,0.008797,11355.527637,0.0999,Sell
180,180,BTC-USD,2020-08-27 00:00:00+00:00,0.008707,11497.094007,0.1001,Buy
181,181,BTC-USD,2020-08-29 00:00:00+00:00,0.034659,11552.595742,0.4004,Buy
182,182,BTC-USD,2020-08-30 00:00:00+00:00,0.034756,11497.205153,0.3996,Sell


In [17]:
pf.value()


symbol,BTC-USD,ETH-USD,XRP-USD,BNB-USD,BCH-USD,LTC-USD
Date,,,,,,
2020-01-01 00:00:00+00:00,17282.817200,17282.817200,17282.817200,17282.817200,17282.817200,17282.817200
2020-01-02 00:00:00+00:00,17282.817200,17282.817200,17282.817200,17282.817200,17282.817200,17282.817200
2020-01-03 00:00:00+00:00,17282.817200,17282.817200,17282.817200,17282.817200,17282.817200,17282.817200
2020-01-04 00:00:00+00:00,17283.505839,17282.817200,17283.048063,17282.817200,17282.817200,17282.817200
2020-01-05 00:00:00+00:00,17283.326416,17282.817200,17284.060703,17282.817200,17282.817200,17282.817200
...,...,...,...,...,...,...
2020-08-27 00:00:00+00:00,21784.630921,27854.808290,19686.257022,22082.232991,16637.074601,19056.006783
2020-08-28 00:00:00+00:00,22198.367259,28624.206756,20078.712244,22126.764989,16774.707411,19408.665512
2020-08-29 00:00:00+00:00,22129.091948,28863.809927,20207.085408,22021.725687,16775.449433,19390.924973


In [18]:
pf.total_profit().vbt.plot()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'name': 'total_profit',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c9487b42-1395-4cab-89fe-2ee01fc6752f',
              'x': array(['BTC-USD', 'ETH-USD', 'XRP-USD', 'BNB-USD', 'BCH-USD', 'LTC-USD'],
                         dtype=object),
              'y': array([ 5172.09602956, 13683.28395566,  3302.9045398 ,  4887.58585593,
                           -336.65716744,  3139.54126666])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [19]:
pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,BTC-USD,2020-01-04 00:00:00+00:00,0.013614,7352.720864,0.1001,Buy
1,1,BTC-USD,2020-01-05 00:00:00+00:00,0.013494,7417.862112,0.1001,Buy
2,2,BTC-USD,2020-01-11 00:00:00+00:00,0.012252,8170.353109,0.1001,Buy
3,3,BTC-USD,2020-01-13 00:00:00+00:00,0.024421,8197.961745,0.2002,Buy
4,4,BTC-USD,2020-01-14 00:00:00+00:00,0.012284,8132.792172,0.0999,Sell
...,...,...,...,...,...,...,...
1069,1069,LTC-USD,2020-08-26 00:00:00+00:00,1.711097,58.383597,0.0999,Sell
1070,1070,LTC-USD,2020-08-27 00:00:00+00:00,1.715304,58.357001,0.1001,Buy
1071,1071,LTC-USD,2020-08-29 00:00:00+00:00,3.481962,57.496312,0.2002,Buy
1072,1072,LTC-USD,2020-08-30 00:00:00+00:00,1.748015,57.264945,0.1001,Buy
